In [1]:

import os
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize
from scipy.io import loadmat

%matplotlib inline

## Vectorized cost function for Regularized Logistic Regression

In [6]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [7]:
def lrCostFunction(theta, X, y, lambda_):
    #Initialize some useful values
    m = y.size
    aux = sigmoid(X@theta)
    theta = theta.copy()
    theta[0] = 0
    
    # convert labels to ints if their type is bool
    if y.dtype == bool:
        y = y.astype(int)
    
    #J:
    J = (-y@np.log(aux) - (1-y)@np.log(1-aux))/m + lambda_*theta@theta/(2*m)

    #grad
    grad = X.transpose()@(aux-y)/m + lambda_*theta/m
        
    return J, grad

## One vs All classification:
Remember that in one vs all classification we are comparing one hypothesis applied to an input to all the other classifications, and we choose the best one.

In [4]:
def oneVsAll(X, y, num_labels, lambda_):
    #We expect X as a matrix, y as a vector and this returns
    #a matrix with all the thetas as rows
    ##
    # Some useful variables
    m, n = X.shape
    
    # You need to return the following variables correctly 
    all_theta = np.zeros((num_labels, n + 1))
    initial_theta = np.zeros(X.shape[1]+1)
    
    # Add ones to the X data matrix
    X = np.concatenate([np.ones((m, 1)), X], axis=1)
    
    options = {'maxiter': 50}
    # ====================== YOUR CODE HERE ======================
    for i in range(num_labels):
        res = optimize.minimize(lrCostFunction, 
                        initial_theta, 
                        (X, (y == i), lambda_), 
                        jac=True, 
                        method='TNC',
                        options=options)
        all_theta[i,:] = res.x

    # ============================================================
    return all_theta


In [5]:
def predictOneVsAll(all_theta, X):
    
    m = X.shape[0];
    num_labels = all_theta.shape[0]
    
    # Add ones to the X data matrix
    X = np.concatenate([np.ones((m, 1)), X], axis=1)

    p = (all_theta@X.transpose()).argmax(axis=0)
    
    return p

## Neural Network:
Here we are going to develop a forward propagation given that we already have a trained neural network. The specific architecture of this neural network has 3 layers. With 400 input layers units, 25 input layers in the second layer and 10 in the output layer.

In [8]:
# Setup the parameters you will use for this exercise
input_layer_size  = 400  # 20x20 Input Images of Digits
hidden_layer_size = 25   # 25 hidden units
num_labels = 10          # 10 labels, from 0 to 9

# Load the .mat file, which returns a dictionary 
weights = loadmat(os.path.join('Data', 'ex3weights.mat'))

# get the model weights from the dictionary
# Theta1 has size 25 x 401
# Theta2 has size 10 x 26
Theta1, Theta2 = weights['Theta1'], weights['Theta2']

# swap first and last columns of Theta2, due to legacy from MATLAB indexing, 
# since the weight file ex3weights.mat was saved based on MATLAB indexing
Theta2 = np.roll(Theta2, 1, axis=0)

In [9]:
def predict(Theta1, Theta2, X):
    # Make sure the input has two dimensions

    if X.ndim == 1:
        X = X[None]  # promote to 2-dimensions
    
    # useful variables
    m = X.shape[0]
    num_labels = Theta2.shape[0]

    # You need to return the following variables correctly 
    p = np.zeros(X.shape[0])

    # ====================== YOUR CODE HERE ======================
    x_1 = np.append(np.ones((m,1)),X,axis=1)
    snd_row = sigmoid(Theta1@x_1.transpose())
    
    w = snd_row.shape[1]
    ones = np.ones(w)[np.newaxis]
    x_2 = np.concatenate([ones,snd_row])
    
    p = sigmoid(Theta2@x_2).argmax(axis=0)
    
    # =============================================================
    return p

pred = predict(Theta1, Theta2, X)
print('Training Set Accuracy: {:.1f}%'.format(np.mean(pred == y) * 100))

Training Set Accuracy: 97.5%


### Main takeaways from Numpy in this lesson:

In [24]:
#array,num_values,replace = True(bootstrap or not),probability of each)
a = np.arange(1,10,2)
np.random.choice(a,2)

array([3, 1])

In [25]:
w = np.arange(10).reshape(2,5)

In [26]:
w

array([[0, 1, 2, 3, 4],
       [5, 6, 7, 8, 9]])

In [27]:
#faster way to transpose: .T
w.T

array([[0, 5],
       [1, 6],
       [2, 7],
       [3, 8],
       [4, 9]])